In [1]:
import os
import gc
import sys
import glob
import time
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import keras
import tensorflow as tf
from keras.utils import plot_model
from keras import regularizers
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten
from keras.layers.merge import Concatenate
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *
from DataLoader import DataLoader

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
step = 3

In [3]:
class NetSetup:
    def __init__(self, activation, dropout_rate, first_layer_size, last_layer_size, decay_factor, kernel_regularizer):
        self.activation = activation
        if activation == 'relu':
            self.DropoutType = Dropout
            self.kernel_init = 'he_uniform'
            self.apply_batch_norm = True
        elif activation == 'selu':
            self.DropoutType = AlphaDropout
            self.kernel_init = 'lecun_normal'
            self.apply_batch_norm = False
        else:
            raise RuntimeError('Activation "{}" not supported.'.format(activation))
        self.dropout_rate = dropout_rate
        self.first_layer_size = first_layer_size
        self.last_layer_size = last_layer_size
        self.decay_factor = decay_factor
        self.kernel_regularizer = kernel_regularizer
    
def add_block_ending(net_setup, name_format, layer):
    if net_setup.apply_batch_norm:
        norm_layer = BatchNormalization(name=name_format.format('norm'))(layer)
    else:
        norm_layer = layer
    activation_layer = Activation(net_setup.activation, name=name_format.format('activation'))(norm_layer)
    if net_setup.dropout_rate > 0:
        return net_setup.DropoutType(net_setup.dropout_rate, name=name_format.format('dropout'))(activation_layer)
    return activation_layer

def conv_block(prev_layer, filters, kernel_size, net_setup, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer=net_setup.kernel_init)(prev_layer)
    return add_block_ending(net_setup, '{}_{{}}_{}'.format(block_name, n), conv)

def reduce_n_features_2d(input_layer, net_setup, block_name):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = net_setup.first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, net_setup, block_name, n)
        if current_size == net_setup.last_layer_size: break
        current_size = max(net_setup.last_layer_size, int(current_size / net_setup.decay_factor))
        n += 1
    return prev_layer

def create_model():
    comp_net_setup = NetSetup('relu', 0.25, 128, 16, 1.4, None)
    #dense_net_setup = NetSetup('relu', 0, 512, 32, 1.4, keras.regularizers.l1(1e-5))
    dense_net_setup = NetSetup('relu', 0.25, 512, 32, 1.4, None)
            
    model_name = "DeepTau2017v2p2"
    input_layer_tau = Input(name="input_tau", shape=(len(input_tau_branches),))
    norm_input_layer_tau = BatchNormalization(name='input_tau_norm')(input_layer_tau)
    input_layers = [ input_layer_tau ]
    
    high_level_features = [ norm_input_layer_tau ]
    for loc in cell_locations:
        reduced_inputs = []
        for comp_id in range(len(component_names)):
            comp_name = component_names[comp_id]
            comp_branches = component_branches[comp_id]
            input_layer_comp = Input(name="input_{}_{}".format(loc, comp_name),
                                     shape=(n_cells_eta[loc], n_cells_phi[loc], len(comp_branches)))
            input_layers.append(input_layer_comp)
            norm_input_layer_comp = BatchNormalization(name='input_{}_{}_norm'.format(loc, comp_name))(input_layer_comp)
            reduced_comp = reduce_n_features_2d(norm_input_layer_comp, comp_net_setup, "{}_{}".format(loc, comp_name))
            reduced_inputs.append(reduced_comp)
        conv_all_start = Concatenate(name="{}_cell_concat".format(loc), axis=3)(reduced_inputs)
        cell_output_size = 32
        prev_layer = conv_block(conv_all_start, cell_output_size, (1, 1), comp_net_setup, "{}_all".format(loc), 1)
        window_size = 4
        current_size = n_cells_eta[loc]
        n = 1
        while current_size > 1:
            win_size = min(current_size, window_size)
            prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), comp_net_setup,
                                    "{}_all_{}x{}".format(loc, win_size, win_size), n)
            n += 1
            current_size -= window_size - 1
            
        cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
        high_level_features.append(cells_flatten)
        
    prev_layer = Concatenate(name="features_concat")(high_level_features)
    current_size = dense_net_setup.first_layer_size
    n = 1
    while True:
        dense_layer = Dense(current_size, name="final_dense_{}".format(n),
                            kernel_initializer=dense_net_setup.kernel_init,
                            kernel_regularizer=dense_net_setup.kernel_regularizer)(prev_layer)
        prev_layer = add_block_ending(dense_net_setup, 'final_{{}}_{}'.format(n), dense_layer)
        n += 1
        if current_size == dense_net_setup.last_layer_size: break
        current_size = max(dense_net_setup.last_layer_size, int(current_size / dense_net_setup.decay_factor))

    output_layer = Dense(n_outputs, name="final_dense_{}".format(n), kernel_initializer=dense_net_setup.kernel_init)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    model = Model(input_layers, softmax_output, name="DeepTau2017v2")
    return model, model_name

In [4]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    if step == 1:
        opt = keras.optimizers.Nadam(lr=learning_rate)
    elif step == 2:
        opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    elif step == 3:
        opt = keras.optimizers.Nadam(lr=learning_rate)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = ["accuracy", TauLosses.tau_crossentropy, TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet, TauLosses.sLe, TauLosses.sLmu, TauLosses.sLjet ]
    if step == 1:
        loss = 'categorical_crossentropy'
    else:
        loss = TauLosses.tau_crossentropy
    model.compile(loss=loss, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [5]:
if step == 1:
    TauLosses.SetSFs(1, 1, 1)
    model, model_name = create_model()
    compile_model(model, 1e-3)
    model.summary()
elif step == 2:
    TauLosses.SetSFs(1, 1, 1)
    model_name = "DeepTau2017v2p2"
    model = LoadModel('DeepTau2017v2p2_step1_historic_final.h5')
    compile_model(model, 1e-5)
elif step == 3:
    TauLosses.SetSFs(1, 5, 1)
    model_name = "DeepTau2017v2p2"
    model = LoadModel('DeepTau2017v2p2_step2_acc.hdf5')
    compile_model(model, 1e-4)    

In [6]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [7]:
class TimeCheckpoint(Callback):
    def __init__(self, time_interval, file_name_prefix):
        self.time_interval = time_interval
        self.file_name_prefix = file_name_prefix
        self.initial_time = time.time()
        self.last_check_time = self.initial_time
    
    def on_batch_end(self, batch, logs=None):
        if batch % 100 != 0: return
        current_time = time.time()
        delta_t = current_time - self.last_check_time
        if delta_t >= self.time_interval:
            abs_delta_t_h = (current_time - self.initial_time) / 60. / 60.
            self.model.save('{}_b{}_{:.1f}h.h5'.format(self.file_name_prefix, batch, abs_delta_t_h))
            self.last_check_time = current_time

In [8]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)
    
    cb_acc = []
    for acc_name in ["acc", "weighted_acc"]:
        cb_acc.append(ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_%s" % acc_name, save_best_only=True,
                                      save_weights_only=False, mode="max", verbose=1))
    
    cb_losses = []
    for loss_name in ["loss", "tau_crossentropy", "Le", "Lmu", "Ljet",
                      "weighted_tau_crossentropy", "weighted_Le", "weighted_Lmu", "weighted_Ljet"]:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    pbar = TQDMNotebookCallback(leave_outer=True, show_outer=True, leave_inner = True)
    time_checkpoint = TimeCheckpoint(60*60, '{}_historic'.format(train_name))
    callbacks = [pbar, time_checkpoint, csv_log, *cb_acc, *cb_losses]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=0)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [9]:
loader = DataLoader('N:/tau-ml/tuples-v2-t2-hdf/training.h5', 300, 2400, validation_size=7491602,
                    max_queue_size=16, n_passes=-1)

print(loader.file_size, loader.data_size, loader.validation_size)

72491602 65000000 7491602


In [ ]:
fit_hist = run_training('step{}'.format(step), model_name, loader, 0, 3)

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.414053). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421849). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421877). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.429545). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421845). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421852). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.423568). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421846). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421905). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421894). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.429705). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421850). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.437498). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.432159). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.435018). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.429457). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.423864). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.429408). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421913). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.424947). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

...

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.464609). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.459715). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.455323). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.467658). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.457348). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.465143). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.471513). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.467728). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.471000). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.460910). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.454078). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.461886). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.472729). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.469628). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.470515). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.470961). Check your callbacks.
  % delta_t_median)
C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update


Epoch 00001: val_acc improved from -inf to 0.95220, saving model to DeepTau2017v2p2_step3_acc.hdf5


In [ ]:
#model.save('p2_l1_try1.h5')